# Daten vorbereiten (*data preprocessing*)

## Bibliotheken installieren

In [31]:
#!pip install numpy
#!pip install pandas
#!pip install matplotlib

## Bibliotheken importieren



In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Daten importieren

Die Daten, die in diesem Beispiel als CSV-Datei vorliegen, wird mithilfe von *pandas* importiert.

In [33]:
dataset = pd.read_csv("data/dataset.csv")

Das geladene Datenset kann hier im Notebook angezeigt werden.

In [34]:
dataset

,Land,Alter,Gehalt,gekauft
0,Frankreich,44.0,72000.0,Nein
1,Griechenland,27.0,48000.0,Ja
2,Deutschland,30.0,54000.0,Nein
3,Griechenland,38.0,61000.0,Nein
4,Deutschland,40.0,NaN,Ja
5,Frankreich,35.0,58000.0,Ja
6,Griechenland,NaN,52000.0,Nein
7,Frankreich,48.0,79000.0,Ja
8,Deutschland,50.0,83000.0,Nein
9,Frankreich,37.0,67000.0,Ja


Für die folgenden Operationen auf den Daten muss das Datenset in zwei Bereiche eingeteilt werden. Dabei sind sind die folgenden Gedanken leitend:

Das zu entwickelnde Modell soll voraussagen können, ob ein Klient Kaufabsichten hat oder nicht. Dafür werden die **Features** `Land`, `Alter` und `Gehalt` zugrundegelegt. Diese Features sind die **unabhängigen Variablen**. Die **abhängige Variable** wiederum ist in der letzten Spalte `gekauft` zu finden. Sie hängt von den Werten der ersten drei Spalten ab.

Es gilt also, ein Modell mit den Features der ersten drei Spalten zu trainieren, um eine Voraussage (*prediction*) für die Kaufabsicht treffen zu können.

Die folgenden Anweisungen sind sehr allgemein formuliert, damit sie für weitere Aufgaben wiederverwendbar sind. Voraussetzung ist, dass die ersten Spalten eines Datensets die **unabhängigen Variablen** enthalten, während die **abhängige Variable** in der letzten Spalte steht.

## Featurematrix auswählen

Zunächst werden die Features in einer Variable zusammengefasst, die einer **Matrix** entspricht.

In [35]:
X = dataset.iloc[:, :-1].values

`iloc` ist eine Methode, die *pandas* für DataFrames zur Verfügung stellt. In den eckigen Klammern wird angegeben, welche Werte einer Tabelle ausgewählt werden sollen. Stehen dort zwei kommaseparierte Werte, bezeichnet der erste die Zeilen, der zweite die Spalten.

:::{.callout-tip}
### Aufgabe
Erklären Sie, was es mit den Doppelpunkten auf sich hat und warum die Angabe für die Spalten mit `-1` notiert ist!
:::

## Abhängigen Vektor bestimmen

Anschließend wird die letzte Spalte als **Vektor** ausgewählt. Diese Werte sollen vorhergesagt werden.

In [36]:
y = dataset.iloc[:, -1].values

Die Anordnung der Features als Matrix kann mit einer einfachen Ausgabe dargestellt werden.

In [37]:
print(X)

[['Frankreich' 44.0 72000.0]
 ['Griechenland' 27.0 48000.0]
 ['Deutschland' 30.0 54000.0]
 ['Griechenland' 38.0 61000.0]
 ['Deutschland' 40.0 nan]
 ['Frankreich' 35.0 58000.0]
 ['Griechenland' nan 52000.0]
 ['Frankreich' 48.0 79000.0]
 ['Deutschland' 50.0 83000.0]
 ['Frankreich' 37.0 67000.0]]


Ebenso die Gestalt der abhängigen Variable als Vektor.

In [38]:
print(y)

['Nein' 'Ja' 'Nein' 'Nein' 'Ja' 'Ja' 'Nein' 'Ja' 'Nein' 'Ja']


## Fehlende Daten berücksichtigen

Wenn wenige Daten fehlen, können diese Datensätze in der Regel ignoriert bzw. entfernt werden. Fehlen in vielen Datensätzen Daten, gibt es verschiedene Strategien, diese zu ersetzen. Eine davon ist, den Mittelwert aus den vorhandenen Daten zu bilden. Hierfür stellt *scikit-learn* ein Verfahren zur Verfügung.

In [39]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])

In [40]:
print(X)

[['Frankreich' 44.0 72000.0]
 ['Griechenland' 27.0 48000.0]
 ['Deutschland' 30.0 54000.0]
 ['Griechenland' 38.0 61000.0]
 ['Deutschland' 40.0 63777.77777777778]
 ['Frankreich' 35.0 58000.0]
 ['Griechenland' 38.77777777777778 52000.0]
 ['Frankreich' 48.0 79000.0]
 ['Deutschland' 50.0 83000.0]
 ['Frankreich' 37.0 67000.0]]


:::{.callout-tip}
### Aufgabe
Recherchieren Sie verwendeten Klassen und Methoden [in der Dokumentation von *scikit-learn*](https://scikit-learn.org/stable/index.html).

:::

## Kategorien in den Daten kodieren

Damit der Algorithmus die Daten schnell und eindeutig verarbeiten kann, müssen Textwerte als Zahlenwerte kodiert werden, hier also die Länder. Ein gängiges Verfahren ist hierbei, binäre Vektoren aus den möglichen Kombinationen der Kategorien zu erstellen. Bei diesem Vorgehen werden neue Spalten angelegt, die diese Vektoren abbilden. [Lesen Sie hierzu auch den Artikel über **One Hot Encoding**, wie dieses Verfahren genannt wird](https://www.educative.io/blog/one-hot-encoding).

Da im vorliegenden Datenset sowohl in den unabhängigen als auch in der abhängigen Variable kategorische Daten vorliegen, ist der Vorgang zweimal notwendig.

### Die unabhängige Variable kodieren

Auch hierfür werden wieder Methoden von *scikit-learn* verwendet.

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [42]:
print(ct)

ColumnTransformer(remainder='passthrough',
                  transformers=[('encoder', OneHotEncoder(), [0])])


In [43]:
print(X)

[[0.0 1.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [1.0 0.0 0.0 40.0 63777.77777777778]
 [0.0 1.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 48.0 79000.0]
 [1.0 0.0 0.0 50.0 83000.0]
 [0.0 1.0 0.0 37.0 67000.0]]


### Die abhängige Variable kodieren

Da die Werte der abhängigen Variablen binär abgebildet werden können, ist hier One-Hot-Encoding nicht notwendig. 

In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [45]:
print(y)

[1 0 1 1 0 0 1 0 1 0]


## Daten aufteilen in Training- und Testset

Für den eigentlichen Prozess des "Maschinenlernens" wird das Datenset in ein **Trainingsset** und ein **Testset** aufgeteilt.

Mit dem Trainingsset wird das Machine-Learning-Modell trainiert, mit dem Testset wird die Vorhersagegüte des Modells getestet. Hierbei wird so getan, als ob es sich bei den Testdaten um zukünftige reale Daten handelt.

*scikit-learn* stellt auch hierfür ein Verfahren bereit.

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [47]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 40.0 63777.77777777778]
 [0.0 1.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 48.0 79000.0]
 [1.0 0.0 0.0 50.0 83000.0]
 [0.0 1.0 0.0 35.0 58000.0]]


In [48]:
print(X_test)

[[1.0 0.0 0.0 30.0 54000.0]
 [0.0 1.0 0.0 37.0 67000.0]]


In [49]:
print(y_train)

[1 0 1 1 0 0 1 0]


In [50]:
print(y_test)

[1 0]


## Feature Scaling

Feature Scaling ist die Anpassung der unabhängigen Daten in einer Weise, dass sie alle im gleichen Verhältnis zueinander stehen. Dadurch wird vermieden, dass ein Feature andere Features dominiert. 

**Wichtig: Feature Scaling wird immer erst nach der Aufteilung des Datensets durchgeführt! Damit wird Informationsverlust bei den Testdaten vermieden.**

### Normalisierung

Hier werden alle Werte in den Bereich zwischen `0` und `1` eingeordnet. Normalisierung wird angewendet, wenn es eine normale Verteilung in den meisten Features gibt.

### Standardisierung

Hierbei werden alle Werte in den Bereich von `-3` und `+3` eingeordnet. Standardisierung funktioniert immer, egal wie verschieden die Verteilung der Werte in den Features ist.

**TODO: Hier noch den Rechenweg**

In [51]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [52]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [1.0 0.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [0.0 1.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [0.0 1.0 0.0 1.1475343068237058 1.232653363453549]
 [1.0 0.0 0.0 1.4379472069688968 1.5749910381638885]
 [0.0 1.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [53]:
print(X_test)

[[1.0 0.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [0.0 1.0 0.0 -0.44973664397484414 0.2056403393225306]]


## Aufgaben

:::{.callout-tip}
### Nachvollziehen der verwendeten Klassen, Methoden und Eigenschaften von *scikit-learn*.
Recherchieren Sie in der Dokumentation von *scikit-learn*, was Sie in diesem Beispiel verwendet haben.
:::

:::{.callout-tip}
### Modell für Vorhersage entwickeln
1. Überlegen Sie, was für ein Modell Sie für diese Daten entwickeln müssen, um die Kaufabsichten von Klienten vorhersagen zu können.
2. Entwickeln Sie das Modell.
:::